In [4]:
%pip install langchain
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2
%pip install accelerate
%pip install -q transformers accelerate langchain
%pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 11.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.165 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[noti

In [2]:
from huggingface_hub import login
login()

In [5]:
from langchain.document_loaders import WikipediaLoader

from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

from datetime import date

In [6]:
with open('api_key.txt', 'r') as file:
    api_key = file.read()

In [7]:
model_repo = 'HuggingFaceH4/zephyr-7b-beta'

llm = HuggingFaceHub(
    repo_id=model_repo,
    task="text-generation",
    huggingfacehub_api_token=api_key,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

model="meta-llama/Llama-2-7b-chat-hf"
tokenizer=AutoTokenizer.from_pretrained(model)
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [8]:
model = ChatHuggingFace(llm=llm)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [30]:
def answer_question_about(person_name,question):

    docs = WikipediaLoader(query=person_name+"\n"+question, load_max_docs=3).load()
    #ext_data = docs[0].page_content[:1000]
    ext_data = [article.page_content[:] for article in docs]

    try:
        system_text_template = """
        Today is: {today}.\n
        You will be given a person name and a question about the person.\n
        Answer the question. Find recent info here: {ext_data}
        """
        human_text_template = "{person_name}, {question}"

        system_message_prompt = SystemMessagePromptTemplate.from_template(system_text_template)
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_text_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])
        prompt = chat_prompt.format_prompt(person_name=person_name, 
                                           today=date.today().strftime("%Y-%m-%d"),
                                           question=question, 
                                           ext_data=ext_data).to_messages()
        answer = model(prompt).content
        return answer.splitlines()[-1]
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [32]:
ask_me = answer_question_about('Vladimir Putin', 'What did he do in 2022?')
print(ask_me)

In 2022, Vladimir Putin ordered and oversaw the Russian invasion of Ukraine, which began on February 24. This military action, known as the Russo-Ukrainian War, has resulted in thousands of casualties and displacement of millions of people. Putin has also faced international sanctions and condemnation for his actions. Additionally, Putin has continued to consolidate power domestically, cracking down on dissent and limiting political freedoms.
